In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

# Assuming df_orders and df_offers are the given DataFrames

# Step 1: Merge the two DataFrames on the 'order_gk' column
merged_df = pd.merge(df_orders, df_offers, on='order_gk', how='left')

# Step 2: Create a new column indicating the reason for failure
# We'll use conditions to determine the reason for failure
merged_df['failure_reason'] = 'No Failure'  # Default value

# Cancellations before driver assignment
merged_df.loc[(merged_df['is_driver_assigned_key'] == 0) & (merged_df['order_status_key'] == 3), 'failure_reason'] = 'Cancellation Before Driver Assignment'

# Cancellations after driver assignment
merged_df.loc[(merged_df['is_driver_assigned_key'] == 1) & (merged_df['order_status_key'] == 3), 'failure_reason'] = 'Cancellation After Driver Assignment'

# Order rejection
merged_df.loc[merged_df['is_driver_assigned_key'] == -1, 'failure_reason'] = 'Order Rejection'

# Step 3: Extract hour from order_datetime and create a new column
merged_df['order_hour'] = pd.to_datetime(merged_df['order_datetime']).dt.hour

# Step 4: One-hot encode the 'failure_reason' column
one_hot_encoded = pd.get_dummies(merged_df['failure_reason'])

# Concatenate the one-hot encoded columns with the original DataFrame
merged_df = pd.concat([merged_df, one_hot_encoded], axis=1)

# Step 5: Filter failed orders and group by hour and reason for failure
failed_orders = merged_df[merged_df['failure_reason'] != 'No Failure']
failed_orders_distribution = failed_orders.groupby(['order_hour']).sum()

# Step 6: Plot the distribution
plt.figure(figsize=(12, 8))
failed_orders_distribution.drop(['order_gk', 'offer_id'], axis=1).plot(kind='bar', stacked=True)
plt.title('Distribution of Failed Orders by Hour and Reason')
plt.xlabel('Hour of the Day')
plt.ylabel('Number of Failed Orders')
plt.xticks(rotation=45)
plt.legend(title='Reason for Failure')
plt.show()

# Step 7: Analyze the results
hourly_failure_totals = failed_orders_distribution.sum(axis=1)
biggest_fail_hour = hourly_failure_totals.idxmax()
highest_failure_count = hourly_failure_totals.max()

print("The hour with the highest number of failed orders is:", biggest_fail_hour)
print("Number of failed orders during this hour:", highest_failure_count)
